In [1]:
import os
from dotenv import load_dotenv
import openai

In [2]:
# Load environment variables from .env file
load_dotenv()

True

In [3]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
from llama_index.core import SimpleDirectoryReader

/Users/abhi/miniconda3/envs/RAG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
documents = SimpleDirectoryReader(
    # input_files = ["./docs/colbert.pdf"]
    input_files = ["./docs/PLAID.pdf", "./docs/colbert.pdf", "./docs/colbert-v2.pdf"]
).load_data()

In [6]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

40 

<class 'llama_index.core.schema.Document'>
Doc ID: 6d6efea0-f160-4225-bd46-52f90b746b92
Text: PLAID: An Efficient Engine for Late Interaction Retrieval Keshav
Santhanam∗ keshav2@stanford.edu Stanford University United StatesOmar
Khattab∗ okhattab@stanford.edu Stanford University United States
Christopher Potts Stanford University United StatesMatei Zaharia
Stanford University United States ABSTRACT Pre-trained language models
are increas...


In [7]:
# print all documents length with a delay i.e. clear the screen after printing each document
# import time
# for doc in documents:
#     print(doc)
#     time.sleep(1)
#     os.system('clear')

In [8]:
len(documents)

40

In [9]:
from llama_index.core import Document

In [10]:
document = Document(text="\n\n".join([doc.text for doc in documents]))

In [11]:
print(document)

Doc ID: a25d9b13-ad25-434f-bd5c-e50f514987ae
Text: PLAID: An Efficient Engine for Late Interaction Retrieval Keshav
Santhanam∗ keshav2@stanford.edu Stanford University United StatesOmar
Khattab∗ okhattab@stanford.edu Stanford University United States
Christopher Potts Stanford University United StatesMatei Zaharia
Stanford University United States ABSTRACT Pre-trained language models
are increas...


In [12]:
from llama_index.core import VectorStoreIndex
from llama_index.core import ServiceContext
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)

/var/folders/0j/d0xd373d2tb7zdzcsqrk3mpw0000gn/T/ipykernel_11599/339795532.py:6: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [13]:
index = VectorStoreIndex.from_documents(
    [document],
    service_context=service_context
    )

In [14]:
query_engine = index.as_query_engine()

In [15]:
response = query_engine.query(
    "explain the query and document encoder. Then show differences between them"
)
print(str(response))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The query encoder preprocesses textual queries by tokenizing them into BERT-based WordPiece tokens and prepending a special token [Q] to indicate it is a query. The input sequence is then passed through BERT's transformer architecture to compute contextualized representations of each token. Query augmentation is applied by padding with BERT's special [mask] tokens to allow for learning to expand or re-weigh terms in the query. The output embeddings are normalized to have an L2 norm of one.

On the other hand, the document encoder segments a document into tokens and prepends BERT's start token [CLS] followed by a special token [D] to indicate it is a document. Unlike queries, no [mask] tokens are appended to documents. After passing through BERT and a linear layer, the document encoder filters out embeddings corresponding to punctuation symbols. This filtering step aims to reduce the number of embeddings per document by removing unnecessary punctuation embeddings.


In [16]:
response = query_engine.query(
    "explain difference between query and document encoder visually"
)

In [17]:
print(str(response))

The query encoder preprocesses textual queries by adding a special token [Q] at the beginning and padding with [mask] tokens if needed before passing the sequence through BERT and a linear layer. On the other hand, the document encoder segments documents into tokens, adds a [CLS] token followed by a special token [D], and filters out embeddings corresponding to punctuation symbols after passing through BERT and the linear layer.


# RAG TRAID

In [18]:
eval_questions = []
with open('eval_questions.txt') as f:
    for line in f:
        eval_questions.append(line.strip())

In [19]:
eval_questions

['How does offline indexing work in ColBert?',
 'What is late interaction?',
 'Explain Query encoder?',
 'Explain Document encoder?',
 'Explain storing document embeddings process?',
 'What is Tok-k re-ranking with colbert?',
 'How does re-ranking works in colbert?',
 'Explain end-to-end retrival with colbert?']

In [39]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

In [40]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
